In [1]:
import glob
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from matplotlib import animation
from scipy.interpolate import Rbf
import pandas as pd
import numba as nb


In [2]:
df = pd.read_csv('./jason3-caspian.csv')


In [3]:
df.head()

,time,lat,lon,MSS,surfacetype,range,ssha,bathymetry
0,6.406609e+08,47.965833,47.882908,-10.8220,3,--,--,-8
1,6.406609e+08,47.922560,47.927560,-10.8742,3,1349699.3757,--,-8
2,6.406609e+08,47.879267,47.972130,-10.8491,3,--,--,-15
3,6.406609e+08,47.835952,48.016619,-10.8157,3,--,--,-19
4,6.406609e+08,47.792616,48.061028,-10.7859,3,--,--,-14


In [4]:
path = "data/JSON-3/JA3*/*.nc"
files = glob.glob(path)
ncid = []
for i, file in enumerate(files):
    ncid.append(nc.Dataset(file))


In [40]:
latMin = 36
latMax = 48
lonMin = 46
lonMax = 55

def read_variable_data(ncid, varname):
    row = []
    for x in range(len(ncid)):
        try:
            row.append(ncid[x].variables[varname])
        except KeyError as e:
            #print(e, len(row), x)
            continue
    return row


def read_pass_number(ncid, varname="lat"):
    row = []
    for x in range(len(ncid)):
        try:
            ncid[x].variables[varname]
            row.append(ncid[x].pass_number)
        except KeyError as e:
            #print(e, len(row), x)
            continue
    return row

In [43]:
latList = read_variable_data(ncid, "lat")
lonList = read_variable_data(ncid, "lon")

dataList = read_variable_data(ncid, "alt_quality_flag")
passNumberList = read_pass_number(ncid)


In [44]:
print(len(passNumberList), len(dataList), len(ncid))

2306 2306 4348


## Numpy attempt

In [47]:
height = len(dataList)
width = len(dataList[0]) * 2

In [46]:
sortedData = []
passNumbers = []
for i in range(len(latList)):
    for j in range(len(latList[i])):
        if (latMax > latList[i][j] > latMin and lonMax > lonList[i][j] > lonMin):
            sortedData.append(dataList[i][j])
            passNumbers.append(passNumberList[i])
    if (i % 100 == 0):
        print("{} of {} elements. Total {}".format(i, len(dataList), len(sortedData)))

0 of 2306 elements. Total 0
100 of 2306 elements. Total 1395
200 of 2306 elements. Total 2668
300 of 2306 elements. Total 4034
400 of 2306 elements. Total 5528
500 of 2306 elements. Total 6881
600 of 2306 elements. Total 8141
700 of 2306 elements. Total 9499
800 of 2306 elements. Total 10970
900 of 2306 elements. Total 12339
1000 of 2306 elements. Total 13838
1100 of 2306 elements. Total 15090
1200 of 2306 elements. Total 16372
1300 of 2306 elements. Total 17843
1400 of 2306 elements. Total 19087
1500 of 2306 elements. Total 20343
1600 of 2306 elements. Total 21691
1700 of 2306 elements. Total 23179
1800 of 2306 elements. Total 24542
1900 of 2306 elements. Total 25804
2000 of 2306 elements. Total 27170
2100 of 2306 elements. Total 28670
2200 of 2306 elements. Total 30026
2300 of 2306 elements. Total 31528


In [ ]:
dataSeries = pd.Series(sortedData)
passSeries = pd.Series(passNumbers)

features = pd.concat([dataSeries, passSeries])
features.columns = ["alt_quality_flag", "pass_number"]

features = pd.concat([df, features])


